In [2]:
import random
import pygame
from pygame.locals import QUIT, KEYDOWN
import time

pygame 2.5.1 (SDL 2.28.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
class Circuit:
    def __init__(self, x, y, width):
        self.x = x
        self.y = y
        self.width = width
        
        self.matrix = None
        
        self.checkpoints = self.generate_checkpoints()
        
        self.create_matrix()
        
    def generate_checkpoints(self):
        checkpoints = []
        mid_x = (self.x + 2 * self.width) // 2
        mid_y = (self.y + 2 * self.width) // 2

        for i in range(1, self.x + 2 * self.width - 1):
            checkpoints.append((i, 1))
            checkpoints.append((i, self.x + 2 * self.width - 2))
            
        for i in range(1, self.y + 2 * self.width - 1):
            checkpoints.append((self.x + 2 * self.width - 2, i))
            checkpoints.append((1, i))

        return checkpoints  
    
    def create_matrix(self):
        circuit = [['-' for _ in range(self.x + 2 * self.width)] for _ in range(self.y + 2 * self.width)]
    
        for i in range(self.width, self.y + self.width):
            for j in range(self.width, self.x + self.width):
                circuit[i][j] = ' '
                if 1 < i < (self.y):
                    if j == (self.x + 2 * self.width) // 2:
                        circuit[i][j] = '-'
                if i == (self.y + 2 * self.width) // 2:
                    if 1 <= j <= (self.x) // 2:
                        circuit[i][j] = '.'
        for checkpoint in self.checkpoints:
            checkpoint_x, checkpoint_y = checkpoint
            if (checkpoint_x != self.x + 2 * self.width - 2) and (checkpoint_y != (self.y + 2 * self.width) // 2):
                circuit[checkpoint_y][checkpoint_x] = 'c'
        self.matrix = circuit
        
    def update_circuit(self, car, window):
        circuit_with_car = [row.copy() for row in self.matrix]
        car_x, car_y = car.get_position()
        circuit_with_car[car_y][car_x] = 'x'
        
        wall_color = (0, 0, 0)
        track_color = (255, 255, 255)
        car_color = (255, 0, 0)
        checkpoint_color = (0, 255, 0)

        cell_size = 40  # Adjust as needed

        for row_index, row in enumerate(circuit_with_car):
            for col_index, cell in enumerate(row):
                cell_rect = pygame.Rect(
                    col_index * cell_size, row_index * cell_size, cell_size, cell_size
                )
                if cell == '-':
                    pygame.draw.rect(window, wall_color, cell_rect)  # Draw wall
                elif cell == ' ':
                    pygame.draw.rect(window, track_color, cell_rect)  # Draw track
                elif cell == 'x':
                    pygame.draw.rect(window, car_color, cell_rect)  # Draw car
                elif cell == 'c':
                    pygame.draw.rect(window, checkpoint_color, cell_rect)  # Draw checkpoint


    def get_circuit(self):
        print('The Circuit')
        for row in self.matrix:
            print(''.join(row))
            
class Car:
    def __init__(self, circuit):
        self.circuit = circuit
        self.x = circuit.x + 2 * circuit.width - 3  # Initial x-coordinate
        self.y = (circuit.y + 2 * circuit.width) // 2  # Initial y-coordinate
        self.ai_controls = []  # Placeholder for AI controls
        
        # Lap counter 
        self.lap_count = 0
        self.lap_line_x = (self.circuit.x + 2 * self.circuit.width) // 2  # X-coordinate of the lap line

    def set_ai_controls(self, genome):
        self.ai_controls = genome

    def move(self):
        prev_x, prev_y = self.x, self.y

        for direction in self.ai_controls:
            self.update_position(direction)
            self.check_collision(prev_x, prev_y)
            prev_x, prev_y = self.x, self.y
        
    def update_position(self, direction):
        if direction == 'w':
            self.y -= 1
        elif direction == 's':
            self.check_lap_completion()
            self.y += 1
        elif direction == 'a':
            self.x -= 1
        elif direction == 'd':
            self.x += 1
        self.check_collision(self.x, self.y)
    
    def update_checkpoint_index(self):
        if self.circuit.matrix[self.y][self.x] == 'c':
            print('suu')
            return True
        return False

    def check_lap_completion(self):
        if (self.circuit.matrix[self.y][self.x] == '.'):
            self.lap_count += 1 
    
    def check_collision(self, prev_x, prev_y):
        if (self.circuit.matrix[self.y][self.x] == '-'):
            self.x = circuit.x + 2 * circuit.width - 2
            self.y = (circuit.y + 2 * circuit.width) // 2

    def get_position(self):
        return self.x, self.y

In [11]:
class CarSimulation:
    def __init__(self, circuit, car, genetic_algorithm):
        self.circuit = circuit
        self.car = car
        self.genetic_algorithm = genetic_algorithm
        self.generation_counter = 0

        pygame.init()
        self.window_width = self.circuit.x / 2 * 100
        self.window_height = self.circuit.y / 2 * 100
        self.window = pygame.display.set_mode((self.window_width, self.window_height))
        pygame.display.set_caption("Car Simulation")

        self.font = pygame.font.Font(None, 36)
        self.clock = pygame.time.Clock()

        self.show_start_screen()

    def show_start_screen(self):
        self.window.fill((0, 0, 0))
        start_text = self.font.render("Press space bar to start the game", True, (255, 255, 255))
        start_rect = start_text.get_rect(center=(self.window_width // 2, self.window_height // 2))
        self.window.blit(start_text, start_rect)
        pygame.display.flip()

        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                    waiting = False


    def run(self, display_every_generations):
        running = True
        generations_since_last_display = 0  # Counter to track generations since last display

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            self.window.fill((0, 0, 0))  # Clear the screen

            best_agent = self.genetic_algorithm.evolve(display_every_generations)
            self.car.set_ai_controls(best_agent.genome)
            self.car.move()

            self.circuit.update_circuit(self.car, self.window)
            pygame.display.update()

            generations_since_last_display += 1
            if generations_since_last_display >= display_every_generations:
                print(f"Generation: {self.genetic_algorithm.current_generation}, Best Fitness: {best_agent.fitness}")
                pygame.time.delay(1000)  # Add a delay of 1 second
                generations_since_last_display = 0

            self.clock.tick(60)  # Limit to 60 FPS

        pygame.quit()

    def evaluate_current_generation(self):
        best_fit = self.genetic_algorithm.evaluate_fitness()
        self.genetic_algorithm.evolve()
        self.generation_counter += 1
        print(f"Generation {self.generation_counter} completed with fitness of {best_fit}.")

    def display_generation_info(self):
        info_text = self.font.render(f"Generation: {self.generation_counter}", True, (255, 255, 255))
        info_rect = info_text.get_rect(topleft=(10, 10))
        self.window.blit(info_text, info_rect)
    
    def display_lap_counter(self):
        lap_text = self.font.render(f"Lap: {self.car.lap_count}", True, (255, 255, 255))
        lap_rect = lap_text.get_rect(topleft=(10, 10))
        self.window.blit(lap_text, lap_rect)

In [12]:
class GeneticAlgorithm:
    def __init__(self, population_size, max_generations):
        self.population_size = population_size
        self.max_generations = max_generations
        self.current_generation = 1
        self.population = []
        self.best_agent = None

    def initialize_population(self):  
        for _ in range(self.population_size):
            initial_genome = [random.choice(['w', 'a', 's', 'd']) for _ in range(genome_length)]
            self.population.append(Agent(initial_genome, Car(circuit)))

    def evaluate_fitness(self):
        best_fit = 0
        for agent in self.population:
            move_completed = agent.car.move()  # Move the car associated with the agent
            if move_completed:
                agent.fitness += 1  # Assign a reward for each move

            lap_completed = agent.car.check_lap_completion()  # Check if lap is completed
            if lap_completed:
                agent.fitness += 10  # Assign a reward for completing a lap
            # Track the best agent
            if self.best_agent is None or agent.fitness > self.best_agent.fitness:
                self.best_agent = agent
            if agent.car.update_checkpoint_index:
                agent.fitness += 3  # Assign a reward for reaching a checkpoint
        
            # check for the higher fitness value
            if agent.fitness > best_fit:
                best_fit = agent.fitness
        return best_fit
        
    def select_parents(self):
        # Select parents for reproduction 
        parent1 = random.choice(self.population)
        parent2 = random.choice(self.population)
        return parent1.genome, parent2.genome

    def crossover(self, parent1_genome, parent2_genome):
        # Perform crossover to create child genome 
        crossover_point = random.randint(1, genome_length - 1)
        child_genome = parent1_genome[:crossover_point] + parent2_genome[crossover_point:]
        return child_genome

    def mutate(self, genome):
        # Perform mutation on genome
        mutated_genome = genome[:]
        mutation_point = random.randint(0, genome_length - 1)
        mutated_genome[mutation_point] = random.choice(['w', 'a', 's', 'd'])
        return mutated_genome

    def evolve(self, display_every_generations):
        self.initialize_population()
        while self.current_generation <= self.max_generations:
            self.evaluate_fitness()
            new_population = []

            while len(new_population) < self.population_size:
                parent1_genome, parent2_genome = self.select_parents()
                child_genome = self.crossover(parent1_genome, parent2_genome)
                child_genome = self.mutate(child_genome)
                new_population.append(Agent(child_genome, Car(circuit)))

            self.population = new_population
            self.current_generation += 1

            if self.current_generation % display_every_generations == 0:
                print(f"Generation: {self.current_generation}, Best Fitness: {self.best_agent.fitness}")

        return max(self.population, key=lambda agent: agent.fitness)
    
    def get_best_agent(self):
        return self.best_agent

class Agent:
    def __init__(self, genome, car):  
        self.genome = genome
        self.fitness = 0
        self.car = car  

In [13]:
# Example usage
genome_length = 10  # Adjust the length as needed
population_size = 50
max_generations = 10000
genetic_algorithm = GeneticAlgorithm(population_size, max_generations)
x = 9
y = 9
width = 1
circuit = Circuit(x, y, width)
car = Car(circuit)
simulation = CarSimulation(circuit, car, genetic_algorithm)
simulation.run(100)

Generation: 100, Best Fitness: 3
Generation: 200, Best Fitness: 3
Generation: 300, Best Fitness: 3
Generation: 400, Best Fitness: 3
Generation: 500, Best Fitness: 3
Generation: 600, Best Fitness: 3
Generation: 700, Best Fitness: 3
Generation: 800, Best Fitness: 3
Generation: 900, Best Fitness: 3
Generation: 1000, Best Fitness: 3
Generation: 1100, Best Fitness: 3
Generation: 1200, Best Fitness: 3
Generation: 1300, Best Fitness: 3
Generation: 1400, Best Fitness: 3
Generation: 1500, Best Fitness: 3
Generation: 1600, Best Fitness: 3
Generation: 1700, Best Fitness: 3
Generation: 1800, Best Fitness: 3
Generation: 1900, Best Fitness: 3
Generation: 2000, Best Fitness: 3
Generation: 2100, Best Fitness: 3
Generation: 2200, Best Fitness: 3
Generation: 2300, Best Fitness: 3
Generation: 2400, Best Fitness: 3
Generation: 2500, Best Fitness: 3
Generation: 2600, Best Fitness: 3
Generation: 2700, Best Fitness: 3
Generation: 2800, Best Fitness: 3
Generation: 2900, Best Fitness: 3
Generation: 3000, Best 